### First we will observe the classical computing case for finding a specfic number in an unsorted list

In [1]:
my_list = [1,3,6,8,2,7,5,7,9,2,16,4]

In [2]:
def my_oracle(my_input):
    winner = 9
    response = False
    if winner == my_input:
        response = True
    return response

In [3]:
for index, number in enumerate(my_list):
    if (my_oracle(number) == True):
        print("Winner winner chicken dinner! at index = %i"%index)
        print("%i times the Oracle was called"%(index+1))
        break

Winner winner chicken dinner! at index = 8
9 times the Oracle was called


### Now, we will apply grover's algorithm through quantum circuits to get the result in SQRT(N) time

In [4]:
from qiskit import *
import matplotlib.pyplot as plt
import numpy as np

In [5]:
# create the oracle quantum circuit
oracle = QuantumCircuit(2, name='oracle')
oracle.cz(0,1) # apply the Controlled-Z gate or CZ gate
oracle.to_gate() # make oracle into its own gate
oracle.draw()

q_0: ─■─
      │ 
q_1: ─■─

In [6]:
backend = Aer.get_backend('statevector_simulator')
grover_circuit = QuantumCircuit(2,2) # 2 qubits, 2 classical registers
grover_circuit.h([0,1]) # apply hadamard gate on both qubits 0 and 1 to prepare superposition state discussed in description.md
grover_circuit.append(oracle, [0,1]) # append oracle to be able to query each state at same time
grover_circuit.draw()

┌───┐┌─────────┐
q_0: ┤ H ├┤0        ├
     ├───┤│  oracle │
q_1: ┤ H ├┤1        ├
     └───┘└─────────┘
c: 2/════════════════

In [7]:
job = execute(grover_circuit, backend)
result = job.result()

In [8]:
sv = result.get_statevector()
np.around(sv, 2)

array([ 0.5+0.j,  0.5+0.j,  0.5+0.j, -0.5+0.j])

In [9]:
reflection = QuantumCircuit(2, name='reflection')
reflection.h([0,1]) # apply hadamard gate on all qubits to bring them back to '00' state from the original 's'  state
reflection.z([0,1]) # apply Z gate on both qubits
reflection.cz(0,1) # controlled-z gate
reflection.h([0,1]) # transform back with hadamard on both qubits
reflection.to_gate()

Instruction(name='reflection', num_qubits=2, num_clbits=0, params=[])

In [10]:
reflection.draw()

┌───┐┌───┐   ┌───┐
q_0: ┤ H ├┤ Z ├─■─┤ H ├
     ├───┤├───┤ │ ├───┤
q_1: ┤ H ├┤ Z ├─■─┤ H ├
     └───┘└───┘   └───┘

In [11]:
backend = Aer.get_backend('qasm_simulator')
grover_circuit = QuantumCircuit(2,2)
grover_circuit.h([0,1])
grover_circuit.append(oracle, [0,1])
grover_circuit.append(reflection, [0,1])
grover_circuit.measure([0,1], [0,1])

In [12]:
grover_circuit.draw()

┌───┐┌─────────┐┌─────────────┐┌─┐   
q_0: ┤ H ├┤0        ├┤0            ├┤M├───
     ├───┤│  oracle ││  reflection │└╥┘┌─┐
q_1: ┤ H ├┤1        ├┤1            ├─╫─┤M├
     └───┘└─────────┘└─────────────┘ ║ └╥┘
c: 2/════════════════════════════════╩══╩═
                                     0  1

In [13]:
job = execute(grover_circuit, backend, shots=1)
result = job.result()
result.get_counts()

{'11': 1}

### we know it worked because it gave us the '11' state as promised, it is just extracted as dictionary key-value pair!

credits to the Qiskit team at IBM (source: https://www.youtube.com/watch?v=0RPFWZj7Jm0&ab_channel=Qiskit)